In [2]:
import pandas as pd 
data=pd.read_csv("data.csv")
data.head()

,鍚屽,鍚屽績,鍚屽織,鍚屾剰,鍚屾瘮,鍚岃垷,鍚岃,鍚岃,鍚岃璺?鍚岄€?鍚岄,鍚嶅北,...,Unnamed: 4539,Unnamed: 4540,Unnamed: 4541,Unnamed: 4542,Unnamed: 4543,Unnamed: 4544,Unnamed: 4545,Unnamed: 4546,Unnamed: 4547,Label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [3]:
x=data.iloc[:,:-1]
y=data.iloc[:,-1]
x.shape

(500, 4548)

In [5]:
from time import time 
import datetime
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, recall_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [6]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(x,y,test_size=0.3,random_state=420)

In [8]:
from sklearn.naive_bayes import MultinomialNB, ComplementNB, BernoulliNB
from sklearn.metrics import brier_score_loss as BS

name = ["Multinomial","Complement","Bournulli"]

models = [MultinomialNB(),ComplementNB(),BernoulliNB()]

In [9]:
import numpy as np
for name,clf in zip(name,models):
    clf.fit(Xtrain,Ytrain)
    y_pred = clf.predict(Xtest)
    proba = clf.predict_proba(Xtest)
    score = clf.score(Xtest,Ytest)
    print(name)
    
    #2个标签下的布里尔分数
    Bscore = []
    for i in range(len(np.unique(Ytrain))):
        bs = BS(Ytest,proba[:,i],pos_label=i)
        Bscore.append(bs)
        print("\tBrier under {}:{:.3f}".format(y[i],bs))
        
    print("\tAverage Brier:{:.3f}".format(np.mean(Bscore)))
    print("\tAccuracy:{:.3f}".format(score))
    print("\n")

Multinomial
	Brier under 0:0.224
	Brier under 1:0.224
	Average Brier:0.224
	Accuracy:0.640


Complement
	Brier under 0:0.222
	Brier under 1:0.222
	Average Brier:0.222
	Accuracy:0.627


Bournulli
	Brier under 0:0.383
	Brier under 1:0.383
	Average Brier:0.383
	Accuracy:0.573




In [10]:
from sklearn.calibration import CalibratedClassifierCV

name = ["Multinomial"
        ,"Multinomial + Isotonic"
        ,"Multinomial + Sigmoid"
        ,"Complement"
        ,"Complement + Isotonic"
        ,"Complement + Sigmoid"
        ,"Bernoulli"
        ,"Bernoulli + Isotonic"
        ,"Bernoulli + Sigmoid"]

models = [MultinomialNB()
          ,CalibratedClassifierCV(MultinomialNB(), cv=2, method='isotonic')
          ,CalibratedClassifierCV(MultinomialNB(), cv=2, method='sigmoid')
          ,ComplementNB()
          ,CalibratedClassifierCV(ComplementNB(), cv=2, method='isotonic')
          ,CalibratedClassifierCV(ComplementNB(), cv=2, method='sigmoid')
          ,BernoulliNB()
          ,CalibratedClassifierCV(BernoulliNB(), cv=2, method='isotonic')
          ,CalibratedClassifierCV(BernoulliNB(), cv=2, method='sigmoid')
         ]

In [11]:
for name,clf in zip(name,models):
    clf.fit(Xtrain,Ytrain)
    y_pred = clf.predict(Xtest)
    proba = clf.predict_proba(Xtest)
    score = clf.score(Xtest,Ytest)
    print(name)
    Bscore = []
    for i in range(len(np.unique(Ytrain))):
        bs = BS(Ytest,proba[:,i],pos_label=i)
        Bscore.append(bs)
        print("\tBrier under {}:{:.3f}".format(y[i],bs))
    print("\tAverage Brier:{:.3f}".format(np.mean(Bscore)))
    print("\tAccuracy:{:.3f}".format(score))
    print("\n")

Multinomial
	Brier under 0:0.224
	Brier under 1:0.224
	Average Brier:0.224
	Accuracy:0.640


Multinomial + Isotonic
	Brier under 0:0.228
	Brier under 1:0.228
	Average Brier:0.228
	Accuracy:0.607


Multinomial + Sigmoid
	Brier under 0:0.220
	Brier under 1:0.220
	Average Brier:0.220
	Accuracy:0.600


Complement
	Brier under 0:0.222
	Brier under 1:0.222
	Average Brier:0.222
	Accuracy:0.627


Complement + Isotonic
	Brier under 0:0.228
	Brier under 1:0.228
	Average Brier:0.228
	Accuracy:0.607


Complement + Sigmoid
	Brier under 0:0.221
	Brier under 1:0.221
	Average Brier:0.221
	Accuracy:0.600


Bernoulli
	Brier under 0:0.383
	Brier under 1:0.383
	Average Brier:0.383
	Accuracy:0.573


Bernoulli + Isotonic
	Brier under 0:0.234
	Brier under 1:0.234
	Average Brier:0.234
	Accuracy:0.627


Bernoulli + Sigmoid
	Brier under 0:0.239
	Brier under 1:0.239
	Average Brier:0.239
	Accuracy:0.540


